## Azure OpenAI Scenarios for Healthcare
<hr>
<hr>

## Disclaimer
You are interacting with Azure OpenAI Service in this chat. Azure OpenAI Service is NOT a clinical decision support medical device.  All responses are for information and general purpose use only and are not designed, intended or made available for the provision of healthcare services.  Any decisions or clinical actions taken on the basis of this chat are solely your responsibility.

## Inroduction

Azure OpenAI Service can be used in virtually every domain imaginable, and its potential in healthcare is especially remarkable. With Azure OpenAI Service, healthcare professionals can now perform tasks such as summarizing clinical notes, extracting entities from notes, performing cohort analysis, creating safety instructional bots, analyzing call center data, and summarizing research papers - all with unprecedented speed and accuracy.

The following use cases have been defined for healthcare scenario: 
- Summarizing Clinical Notes
- Extracting Entities from Notes
- Cohort Analysis
- Safety Instructional Bot
- Call Center Analytics
- Summarizing Medical Journals 

## Contents

The demo will be divided into five sections, each having its own sub section

**0. Setting up Notebook**

    0.1 Installation

    0.2 Imports

    0.3 Configuration

    0.4 Brief Description of Model Parameters
<hr>

**1. Working with clinical notes**

    1.1 Generating Contextual Summary

        1.1.C Code

    1.2 Extracting Entity   
        
        1.2.1 Extracting Medication

            1.2.1.C Code

        1.2.2 Extracting Vitals

            1.2.2.C Code

    1.3 Sentiment Analysis

        1.3.C Code
<hr>

**2. Cohort Analysis**

    2.C Code
<hr>

**3. Global Occupational Safety**

    3.C Code
<hr>

**4. Call Center Analytics**

    4.1 Script Improvement

        4.1.C Code

    4.2 Track Issue Resolution and Extract Summary

        4.2.C Code
<hr>

**5. Summarizing Medical Journals **

    5.C Code
<hr>

## 0 Setting up Notebook

### 0.1 Installations

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


### 0.2 Imports

In [2]:
import openai
import os
import numpy as np
import pandas as pd
from openai import cli
import json
print("Module Importing Successful")

Module Importing Successful


### 0.3 Configuration

In [3]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the deployment name
deployment_name = config_details['COMPLETIONS_MODEL_DV']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
API_KEY = config_details['OPENAI_API_KEY']
openai.api_key = os.getenv(API_KEY)
openai.api_key = API_KEY

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']

ENGINE = config_details['OPENAI_ENGINE_DV']

print("configuration complete")
print("----------------------")
print("Model      : " + str(deployment_name))
print("Deployment : " + str(ENGINE))
print("Endpoint   : " + str(config_details['OPENAI_API_BASE']))
print("----------------------")

configuration complete
----------------------
Model      : text-davinci-003
Deployment : text-davinci-003
Endpoint   : https://chatgpttest45.openai.azure.com/
----------------------


### 0.4 Brief Description of Model Parameters

The parameters we have taken in this are changed and they mean the follows:

1. **prompt** - The trigger text that we to give to the model
2. **temperature** - The amount of randomness, since we need clear cut answers we have kept it as 0
3. **max_tokens** - Length of the answer given
4. **top_p** - distribution of probability of common tokens. 1.0 means "use all tokens in the vocabulary" 
5. **frequency_penalty**  and **presence_penalty** - controls the models tendency to use the same predictions, we dont want it to talk about new things so we have kept it as 0

## 1. Working with Clinical Notes

### Introduction

The patient's clinical records document their medical history and must be maintained to include all relevant medical records, progress notes, charts, admission and discharge data, and any other information pertaining to the patient's hospitalization or treatment. This can benefit the patient by reducing the time spent repeating tests, and can benefit healthcare organisations by allowing them to make decisions for a single patient more quickly, freeing up time to spend with those most in need. In this example, a sample of clinical records has been taken, which includes patient demographics, medical history, laboratory test results, medications, imaging studies, vital signs, progress notes, and discharge summaries. This data can be used to identify trends in patient care, track the effectiveness of treatments, and identify potential areas for improvement. It can also be used to develop evidence-based protocols for patient care and improve patient outcomes. Healthcare organisations can use the data from clinical records to provide better care for their patients and improve the overall quality of care.

In this section we will extract the entity and summarise the clinical records.

### 1.1 Generating Contextual Summary

#### Brief Description

Text summarization is a technique for automatically creating a short, accurate, and fluent summary of a longer text document. It is used in natural language processing (NLP) to reduce the length of a text by extracting the most important information from it. Text summarization can be done using a variety of algorithms, including natural language processing, semantic analysis, and statistical and machine learning techniques.

Text summarization can be very beneficial in terms of saving time and energy. By summarizing a text, readers can quickly and efficiently get the gist of a document without having to read the entire thing. This can be especially useful for lengthy documents, such as research papers or news articles. Summarization can also be helpful for detecting patterns, such as in text mining or sentiment analysis. Additionally, text summarization can help improve the accuracy of search engines by providing more concise search results.

Generating Contextual summaries help increase patients' recall and comprehension by providing them with relevant and actionable health information to refer to after their visit. Clinical summaries improve patients' quality of care and offer a way for patients and their families to stay informed and engaged.

##### 1.1.C. Code

In [4]:
text1 = """
# Chief Complaint No complaints.  
# History of Present Illness Reta is a 30 year-old female (race undeclared). 
Patient has a history of acute bacterial sinusitis (disorder), fatigue (finding), headache (finding), viral sinusitis (disorder), sprain of wrist, 
antepartum eclampsia, suspected covid-19, covid-19, normal pregnancy, fever (finding), sputum finding (finding), blighted ovum, preeclampsia, anemia (disorder), cough (finding), second degree burn, loss of taste (finding).  
# Social History Patient is single. Patient is an active smoker and is an alcoholic. 
Patient identifies as heterosexual.   Patient comes from a high socioeconomic background.  Patient is a college graduate.  Patient currently has Medicaid.  
# Allergies  No Known Allergies.  # Medications  natazia 28 day pack; cefuroxime 250 mg oral tablet; seasonique 91 day pack; trinessa 28 day pack; meperidine hydrochloride 50 mg oral tablet; naproxen sodium 220 mg oral tablet;   
# Assessment and Plan   ## Plan  The following procedures were conducted:  - intramuscular injection The patient was prescribed the following medications:  - 1 ml medroxyprogesterone acetate 150 mg/ml injection   
# Vital Sign: Vitals recorded for patient at the arrival were Blood Pressure was observed as 117/79 mm[Hg],Body temperature was observed as 35.2 C, Carbon dioxide [Partial pressure] in Blood was observed as 4.7 kPa, Erythrocytes [#/volume] in Blood by Automated count was observed as 4.4 10*12/uL, eye color was observed as blue, Gender identity was observed as Feminism, Glucose [Moles/volume] in Blood was observed as 3.2 mmol/l, Heart rate was observed as 65 /min, Hemoglobin [Mass/volume] in Blood was observed as 8.6 g/dL, Oxygen saturation in Arterial blood by Pulse oximetry was observed as 91%, Second hand smoke exposure was observed as YES.
"""

In [5]:
def summarise_text(prompt):
    len_org = len(prompt.split(" "))
    max_t = int(40/100 * len_org)  
    instruction = "perform text summarization \n\n: "
    prompt = instruction + prompt
    response = openai.Completion.create(
        engine=ENGINE,
        prompt= prompt,
        temperature=0.9,
        max_tokens=max_t,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None)

    print("original length: ",len_org)
    
    res = response['choices'][0]['text']

    print("summarized length: ",len(res.split(" ")))
    
    print(res)

In [6]:
summarise_text(text1)

original length:  286
summarized length:  48
Patient has a history of multiple disorders and was given various medications, intramuscular injections, and her vitals were recorded at arrival. There are no known allergies and the patient identifies as feminism, is a college graduate, an active smoker and alcoholic, and is from a high socioeconomic background.


### 1.2 Entity Extraction

#### Brief Description

Entity extraction is a process of automatically extracting structured information from unstructured text. It is used in natural language processing (NLP) to identify and classify the entities mentioned in a text document. The entities can be people, organizations, locations, products, and so forth. Entity extraction is used in a variety of applications, such as question answering systems, sentiment analysis, and text mining.

Entity extraction can be very beneficial in terms of saving time and energy. By extracting entities from a text, readers can quickly and efficiently get the important information from it. This can be especially useful for lengthy documents, such as research papers or news articles. Entity extraction can also be helpful for detecting patterns, such as in text mining or sentiment analysis. Additionally, entity extraction can help improve the accuracy of search engines by providing more accurate search results.

#### 1.2.1 Extracting Medication

Medication is a medicine or a chemical compound used to treat or cure illness. Medications come in many dosage forms, including tablets, capsules, liquids, creams, and patches. Medication is a substance used to treat, cure, prevent, or diagnose a disease or to enhance physical or mental well-being. It can be a prescription drug, an over-the-counter drug, a herbal remedy, or another type of substance, such as vitamins or supplements.

##### 1.2.1.C Code

In [7]:
def extract_entity_medication(prompt):  
  instruction = "perform entity extraction for Medicine Name, Dosage, Duration, Frequency, Route in table format \n\n: "
  prompt = instruction + prompt
  response = openai.Completion.create(
      engine=ENGINE,
      prompt=prompt,
      temperature=0,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      best_of=1,
      stop=["\n\n"],
  )
  res = response['choices'][0]['text']
  print(res)

In [8]:
extract_entity_medication(text1)


Medicine Name | Dosage | Duration | Frequency | Route 
natazia 28 day pack | - | 28 days | - | intramuscular injection 
cefuroxime 250 mg oral tablet | 250 mg | - | - | oral 
seasonique 91 day pack | - | 91 days | - | - 
trinessa 28 day pack | - | 28 days | - | - 
meperidine hydrochloride 50 mg oral tablet | 50 mg | - | - | oral 
naproxen sodium 220 mg oral tablet | 220 mg | - | - | oral 
medroxyprogesterone acetate 150 mg/ml injection | 1 ml | - | - | intramuscular injection


#### 1.2.2 Extracting Vitals

Vital are used to measure the body's basic functions. These measurements helps to assess the general physical health of a person which gives clues to possible diseases and show progress toward recovery. The normal ranges for a person's vital signs vary with age, weight, gender and overall health.The four main vital signs are body temperature, blood pressure, pulse (heart rate), and breathing rate.

Vital signs are important indicators of a patient's health and are used to assess the patient's condition and any changes in the patient's condition. They provide important information for making medical decisions, as well as for helping to identify medical emergencies. Vitals are also used to monitor the effectiveness of treatments. By taking and recording vitals regularly, healthcare providers can identify any changes in the patient’s condition and provide timely intervention if necessary.

##### 1.2.2.C Code

In [9]:
def extract_entity_vitals(prompt):  
  instruction = "perform entity extraction for Lab test name, lab test result in table format  \n\n: "
  prompt = instruction + prompt
  response = openai.Completion.create(
      engine=ENGINE,
      prompt=prompt,
      temperature=1,
      top_p= 0,
      max_tokens=1000,
      best_of=1,
      stop=None,
  )
  res = response['choices'][0]['text']
  print(res)

In [10]:
extract_entity_vitals(text1)


Lab Test Name | Lab Test Result 
------------ | ---------------
Blood Pressure | 117/79 mm[Hg]
Body temperature | 35.2 C
Carbon dioxide [Partial pressure] in Blood | 4.7 kPa
Erythrocytes [#/volume] in Blood by Automated count | 4.4 10*12/uL
Glucose [Moles/volume] in Blood | 3.2 mmol/l
Heart rate | 65 /min
Hemoglobin [Mass/volume] in Blood | 8.6 g/dL
Oxygen saturation in Arterial blood by Pulse oximetry | 91%


### 1.3 Sentiment Analysis

#### Brief Description

Sentiment analysis is a process of analyzing a given text or set of texts to determine the underlying sentiment or attitude expressed by the author. It is used to gain an understanding of the attitudes, opinions, and emotions expressed in a text by identifying and extracting relevant keywords and phrases. The goal of sentiment analysis is to determine whether the author expresses a positive, negative, or neutral sentiment.

Sentiment analysis of clinical records is important because it allows healthcare providers to gain insights into a patient’s emotional state and thoughts, which can help inform diagnosis and treatment decisions. It can also help healthcare providers to better understand a patient’s experience and communicate more effectively with them. Furthermore, sentiment analysis can be used to track changes in a patient’s emotional state over time, which can be useful for monitoring the effectiveness of treatments and interventions.

##### 1.3.C Code

In [11]:
def get_sentiment(prompt):  
  instruction = "perform sentiment Analysis \n\n: "
  prompt = instruction + prompt
  response = openai.Completion.create(
      engine=ENGINE,
          prompt= prompt,
          temperature=0.9,
          max_tokens=1000,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          best_of=1,
          stop=None,
  )
  res = response['choices'][0]['text']
  print(res)

In [12]:
get_sentiment(text1)


The sentiment of this analysis is mostly neutral, though there appears to be some concern for the patient's health due to the presence of various medical issues and lifestyle choices. There does not appear to be any outward judgement or negativity.


## 2. Cohort Analysis

### Introduction

Cohort analysis is a subset of behavioral analytics that takes the data from a given dataset and groups it into related groups, or cohorts, for the purpose of observational study. This type of analysis is typically used to measure user behavior over time, such as how long it takes users to complete a purchase or how often a customer returns. It can also be used to compare different cohorts, such as those who joined a service at different times or those who have different demographic characteristics.

In this scerio:
- Given a context, we want the engine to answer based on the question given. The problem with asking the GPT3 models without any context is that it will try to answer even if it does not have answers. This is a problem, and to eliminate it we will fine tune the model prompt.

##### 2.C Code

In [13]:
instruction = """
    Question: Hi
    Hello, How can I help you ! \n

    Here’s a cohort of all the people who have reported some illness with Contoso Healthcare. Based on the data available in this report, there are a total of 14,195 patients, of which 55% have reported shortness of breath.

    Of these 55% patients who reported shortness of breath, 34.58% are confirmed with the diagnosis, whereas 65.42% have shown atypical symptoms. Most of the patients with confirmed diagnosis belong to Gen X and are between the age group 31-40 years, followed by the age group 51 – 60 years. When ranked based on their city, Miami has reported the highest number of patients with Honolulu at second place, followed by Anchorage, Chicago, and Los Angeles.

    These patients have also reported some other attributes, with Obesity being the most common, being less active as the second and smoking the third. Along with shortness of breath, they have also shown other conditions such as Allergies, Asthma, Bronchitis, and Heart Failure.

    Let’s look at the key influencing factors when the patient is more likely to have a confirmed diagnosis. This patient cohort profile can be decomposed into 6 segments. The first segment of people is 94.9% likely to have confirmed diagnosis because they are Obese, less active, smokers and belong to Gen X, which is within the age group of 31-40 years. Also, if they reside in Miami, a city with poor Air Quality Index, they are more vulnerable to shortness of breath.

    Based on the Air quality Prediction for the next 5 days for Miami, the Providers are recommended to prepare for a 20% increase in regular bed occupancy as more people are likely to be admitted for shortness of breath. It is also advised to send a notification to this cohort in advance so that they can take precautionary measures to avoid health risks.

    A sample communication template might be useful to send a quick push notification to this cohort.

    ALERT: Poor air quality is expected in the next 10 days. Consult your physician for information on precautionary measures you can take to avoid health risks.\n: """

In [14]:
def PatientCohort_bot(prompt):  

  prompt = instruction + prompt
  response = openai.Completion.create(
      engine=ENGINE,
      prompt=prompt,
      temperature=0,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      best_of=1,
      stop=None)
  res = response['choices'][0]['text']
  print(res)

In [15]:
prompt1 = "What is the recommendation for Providers based on the information from this cohort?"
prompt2 = "Any recommendations for communications to this cohort?"
prompt3 = "Can you share a sample communication letter for this patient?"
prompt4 = "What is recommendation for an Asthama patient aged 60"

prompt5 = "What age group does Gen X belong to?"
prompt6 = "What are the other health conditions found in this cohort?"
prompt7 = "What is the most common attribute among these patients?"

In [16]:
print(prompt2)
PatientCohort_bot(prompt2)

Any recommendations for communications to this cohort?


Yes, it is recommended to send a notification to this cohort in advance so that they can take precautionary measures to avoid health risks. A sample communication template might be useful to send a quick push notification to this cohort. The template should include information about the expected poor air quality, the potential health risks, and the recommended precautionary measures. Additionally, it is important to provide contact information for the patient's physician in case they have any questions or concerns.


In [17]:
print(prompt3)
PatientCohort_bot(prompt3)

Can you share a sample communication letter for this patient?


Dear [Name],

We are writing to inform you that the air quality in your area is expected to be poor in the next 10 days. Poor air quality can lead to health risks, especially for those with existing conditions such as shortness of breath.

We recommend that you consult your physician for information on precautionary measures you can take to avoid health risks.

If you have any questions or concerns, please do not hesitate to contact us.

Sincerely,

[Name]
Contoso Healthcare


In [18]:
print(prompt4)
PatientCohort_bot(prompt4)

What is recommendation for an Asthama patient aged 60
?

For an asthmatic patient aged 60, it is recommended to take precautionary measures to avoid health risks. This includes avoiding exposure to air pollutants, exercising regularly, and taking medications as prescribed by a physician. Additionally, it is important to monitor symptoms and seek medical attention if necessary.


### 3. Global Occupational Safety

#### Introduction

Global Occupational Safety and Health (GOSH) is the application of safety and health principles, standards, and best practices to protect workers in all parts of the world. It is an international effort to ensure that all workers, regardless of where they live, have access to safe and healthy working conditions. GOSH encompasses a wide range of topics, including workplace safety, hazardous materials, ergonomics, and health promotion.

A bot is a software application that is designed to automate certain tasks. Bots often mimic or simulate human behavior, performing actions such as responding to user input or initiating conversations with other users. A contextual bot is an AI-powered chatbot that uses machine learning and natural language processing to understand the context of a conversation and respond accordingly. Contextual bots are able to learn from conversations over time, allowing them to offer more personalized, relevant, and accurate answers.

In this scenario a bot is being created to help employees with GOSH.

##### 3.C Code

In [19]:
instruction = """
    According to the Global Occupational Safety Report of Contoso Healthcare which is
located in different locations like Anchorage, Chicago, Honolulu, Los Angeles and
Miami reported 161 Incidents. 
1.In Anchorage the leading source of injury from accidents is fractures followed
by unspecified injuries, strain, trauma, respiratory symptoms, and allergic
reactions. The main reason for such injuries is mostly stairs and the number
of accidents reported there is 32.
2.In Chicago the leading source of injury from accidents is unspecified followed
by fractures, hernia, allergic reactions and multiple poisoning, toxic, noxious,
allergenic effects. The major causes of such injuries are disinfectants,
vaccines and stairs and the number of accidents reported there is 30.
3.In Honolulu the leading source of injury from accidents is unspecified injury
followed by fractures, Bruises, contusions, muscle tears, hernia, and strain.
The causes of these are indoor stairs and the number of accidents reported
there is 42.
4.In Los Angels the leading injuries from accidents is fractures followed by
hernia, unspecified injuries, allergic reaction, and concussion. The cause of
these is mainly due to the stairs and the number of accidents reported there is
29.
5.In Miami the leading injuries from accidents are fractures followed by
unspecified injuries, poisoning or allergies. The reason for these accidents is
stairs, vaccines, smoke, fire gases and n.e.c and the number of accidents
reported there is 28
    """

In [20]:
def QA_bot(prompt):  
    prompt = instruction + prompt
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=0.9,
        max_tokens=1500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)

In [21]:
prompt1 = "What is the leading cause of injuries in Miami?"
prompt2 = "What is the leading injury in Chicago?"
prompt3 = "How many hospitals are there in Anchorage?"
prompt4 = "What is the second highest injury in Chicago?"
prompt5 = "Which city reports the highest number of accidents?"

In [22]:
print(prompt1)
QA_bot(prompt1)

What is the leading cause of injuries in Miami?


The leading cause of injuries in Miami is stairs.


In [23]:
print(prompt5)
QA_bot(prompt5)

Which city reports the highest number of accidents?


The city that reports the highest number of accidents is Honolulu with a total of 42 accidents reported.


## 4. Call Centre Analytics

### Introduction

Call center analytics is the process of using data and technology to analyze the performance of a call center. It helps organizations optimize operations, improve customer experience, and increase revenue. Call center analytics can be used to analyze customer interactions, such as call volume, length of calls, customer satisfaction ratings, and customer feedback. It can also be used to identify trends and opportunities, such as increasing call volumes or identifying areas of improvement in customer service. By leveraging call center analytics, organizations can ensure that their customer service teams are performing at their best and that customers are receiving the best possible experience.

 It is also used to identify trends and opportunities, such as increasing call volumes or customer service areas that need improvement. With call center analytics, organizations can ensure that their customer service teams are performing at their best and that customers are receiving the best possible experience. The analytics also provide valuable insights on customer behavior and preferences that can be used to customize services and improve customer loyalty.

#### Transcript

A transcript in a call center is a written record of a customer's support experience. It documents all the customer's interactions with customer service agents, such as their questions, concerns, and comments A transcript typically includes the customer's name, contact details, and other pertinent information. Call center transcripts are important for providing customer service agents with the appropriate context to address a customer’s needs. They can also be used to monitor customer service performance and identify areas for improvement.

##### 4.C Code

In [24]:
transcript = """
Hospital staff: Good morning, this is the hospital. How may I help you?
Patient: Yes, I am having some medical issues and was wondering if I could come in to get checked out.
Hospital staff: Certainly. What are the symptoms you are experiencing?
Patient: I have a fever, aches and chills, and a sore throat.
Hospital staff: Okay, it sounds like you may have the flu. We can schedule you in to see a doctor if you’d like.
Patient: That would be great. What time would be best for me to come in?
Hospital staff: We have an opening at 1 pm this afternoon. Would that work for you?
Patient: Yes, that would be perfect.
Hospital staff: Alright, I will go ahead and book that appointment for you. Please arrive about 15 minutes before your appointment so the receptionist can get you checked in.
Patient: Got it. Can I bring someone with me to the appointment?
Hospital staff: Yes, you can bring a friend or family member with you.
Patient: Alright, thank you.
"""

### 4.1 Script Improvement


#### Brief Description

Transcript improvement is a process of making transcripts of audio or video recordings more accurate and easier to read. This process can involve a variety of techniques, such as automatic speech recognition, editing and formatting of text, and audio and video synchronization. The goal of transcript improvement is to produce a document that accurately reflects the original recording in a way that is easy for readers to understand. This can help to improve the accuracy of audio and video recordings, as well as make them easier to use in court proceedings and other legal proceedings.

##### 4.1.C Code

In [25]:
instruction ="improve the above transcript:"
def transcript_betterment(prompt,instruction):
    prompt = prompt + instruction
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=0.9,
        max_tokens=2500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)
    
transcript_betterment(transcript,instruction)



Hospital staff: Good morning, this is the hospital. How may I help you?
Patient: Yes, I am having some medical issues and was wondering if I could come in to get checked out.
Hospital staff: Certainly. What are the symptoms you are experiencing?
Patient: I have a fever, aches and chills, and a sore throat.
Hospital staff: Okay, it sounds like you may have the flu. We can schedule you in to see a doctor if you’d like. When would be the best time for you?
Patient: This afternoon would be great.
Hospital staff: We have an opening at 1 pm this afternoon. Would that work for you?
Patient: Yes, that would be perfect.
Hospital staff: Alright, I will go ahead and book that appointment for you. Please arrive about 15 minutes before your appointment so the receptionist can get you checked in. Is there anything else I can help you with today?
Patient: Yes, can I bring someone with me to the appointment?
Hospital staff: Yes, you can bring a friend or family member with you.
Patient: Alright, tha

### 4.2 Track Issue Resolution and Extract Summary

#### Brief Description

**Issue Resolution**

Issue resolution for a call center is the process of resolving customer complaints and inquiries. This involves listening to the customer, understanding their needs, and providing solutions that meet their expectations. It requires both technical expertise and interpersonal skills. The goal of issue resolution is to ensure customer satisfaction and loyalty, as well as to improve customer experience.

Issue resolution for a call center requires representatives to be knowledgeable about the company's products and services, be able to accurately assess customer needs, and be patient and understanding when addressing customer complaints. They must also be able to effectively listen to customers and provide them with clear explanations and solutions. Representatives should also be able to accurately document customer complaints and inquiries and provide feedback to the customer. Finally, representatives should be able to provide follow-up support and ensure that the customer's experience has been satisfactory.

**Summarization of Transcript**

Summarization of a call transcript is the process of taking a long, detailed conversation and condensing it into a shorter, more concise summary. This summary can be useful for businesses that need to quickly and easily recall the main points of a conversation or to provide a brief overview of a conversation for future reference. The benefit of summarizing a call transcript is that it can provide a quick, easily digestible snapshot of the key points of a conversation, saving time and resources.
Summarizing a call transcript involves taking a long, detailed conversation and turning it into a shorter, more concise summary. This summary should cover the main points of the conversation, including any key decisions that were made and any important information that was exchanged. The summarization should also include any action items that were agreed upon and any follow-up steps that need to be taken. By creating a concise summary, businesses can quickly and easily recall the main points of a conversation and have a brief overview of a conversation for future reference. This can save time and resources when reviewing calls and can help to ensure that important points are not forgotten.

##### 4.2.C Code

In [26]:
def get_resolution(text):
    add_text = """Generate a summary of the below given conversation in the following format:
                  Customer problem:
                  Outcome of the conversation:
                  Action items for follow-up::
                  Medication Given (if any):
                  Summary of Call:\n"""
    prompt = add_text + '\n' + text + '\n' + 'Summary:' + '\n'
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=0.7,
        max_tokens=1500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)

In [27]:
get_resolution(transcript)

Customer problem: Patient is experiencing a fever, aches, chills, and a sore throat.
Outcome of the conversation: Patient is scheduled for an appointment at 1 pm this afternoon.
Action items for follow-up: Patient to arrive 15 minutes before the appointment. Patient can bring a friend or family member to the appointment.
Medication Given (if any): None.
Summary of Call: Patient is scheduled for an appointment at 1 pm this afternoon. The patient was advised to arrive 15 minutes prior to the appointment, and can bring a friend or family member with them.


## 5. Summarizing Medical Journals 

### Introduction

Medical Journals summarization is the process of condensing a journal into a shorter, more concise form. This is often done by removing unnecessary details, shortening long passages, and eliminating irrelevant content. The goal of summarization is to present the main points of the paper in a way that is easily understood.

The benefit of summarizing a journal is that it allows readers to quickly understand the main points and figures of the journal without having to read the entire document. This saves time and allows readers to focus on the most important parts of the research. Summarization also allows readers to quickly determine if the paper is relevant to their interests or if they should move on and read other papers. Summarizing a journal can also help the reader better understand the journal's arguments and points and can provide a quick reference to the journal's main ideas.

##### 5.C Code

In [28]:
def get_summary(corpus):
    import nltk
    nltk.download('punkt')
    from nltk.tokenize import sent_tokenize

    def split_text_into_paragraphs(text):
        # Split the text into sentences.
        sentences = sent_tokenize(text)
        # Split the sentences into small paragraphs of around 10000 tokens each.
        paragraphs = {}
        current_paragraph = ''
        current_length = 0
        current_key = 1
        for sentence in sentences:
            # If adding the current sentence to the current paragraph exceeds the target length,
            # add the current paragraph to the dictionary of paragraphs and start a new paragraph.
            if current_length + len(sentence) + 1 > 10000:
                while current_paragraph[-1] not in ['.', '!', '?']:
                    current_paragraph = current_paragraph.rsplit(' ', 1)[0]
                paragraphs[f'para{current_key}'] = current_paragraph.strip()
                current_paragraph = ''
                current_length = 0
                current_key += 1
            # Add the current sentence to the current paragraph.
            current_paragraph += sentence + ' '
            current_length += len(sentence) + 1
        # Add the last paragraph to the dictionary of paragraphs.
        while current_paragraph[-1] not in ['.', '!', '?']:
            current_paragraph = current_paragraph.rsplit(' ', 1)[0]
        paragraphs[f'para{current_key}'] = current_paragraph.strip()
        
        return paragraphs

    def summarise_text(corpus):
        len_org = len(corpus.split(" "))
        max_t = int(40/100 * len_org)  
        instruction = "Generate Summary in third person: "
        prompt = instruction + corpus
        response = openai.Completion.create(
            engine=ENGINE,
            prompt= prompt,
            temperature=0.9,
            max_tokens=max_t,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            best_of=1,
            stop=None)  
        res = response['choices'][0]['text']
        return res


    def summarise_universal_text(corpus):
        instruction = "Generate high level summary from following text:"
        prompt = instruction + corpus
        response = openai.Completion.create(
            engine=ENGINE,
            prompt= prompt,
            temperature=0.9,
            max_tokens=1000,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            best_of=1,
            stop=None)  
        res = response['choices'][0]['text']
        return res
    
    paragraphs = split_text_into_paragraphs(corpus)
    print("Paragraphs Divided")
    universal_summary = []
    for para in paragraphs:
        individual_summary = summarise_text(paragraphs[para])
        universal_summary.append(individual_summary)

    txt = ""
    for summary in universal_summary:
        txt = txt + summary

    txt =  txt.replace('\n', '')
    print("Summary:")
    us = summarise_universal_text(txt)
    print(us)
    return paragraphs,txt


<hr>
End of Document
<hr>